In [5]:
## 스네이크 게임 import 코드

import pygame
import os
import sys
import random
from time import sleep


## 게임 화면 전역변수

# 게임 스크린 전역변수
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# 게임 화면 전역변수
GRID_SIZE = 20
GRID_WIDTH = SCREEN_WIDTH / GRID_SIZE
GRID_HEIGHT = SCREEN_HEIGHT / GRID_SIZE


## 방향 전역변수
UP = (0, -1)
DOWN = (0, 1)
LEFT = (-1, 0)
RIGHT = (1, 0)
STOP = (0, 0)


## 색상 전역변수
WHITE = (255, 255, 255)
ORANGE = (250, 150, 0)
GRAY = (100, 100, 100)
RED = (255, 0, 0)


## 뱀 객체 선언

# 뱀 객체
class Snake():
    
    die = False
    
    ## 뱀 객체 초기화 __init__() 메서드
    def __init__(self):
        self.create()


    ## 뱀 객체 create() 메서드

    # 뱀 생성
    def create(self):
        self.die = False
        self.length = 2
        self.positions = [(int(SCREEN_WIDTH / 2), int(SCREEN_HEIGHT / 2))]
        self.direction = random.choice([UP, DOWN, LEFT, RIGHT])


    ## 뱀 객체 control() 메서드

    # 뱀 방향 조절
    def control(self, xy):
        if not self.die:
            if (xy[0] * -1, xy[1] * -1) == self.direction:
                return
            else:
                self.direction = xy
        else :
            self.direction = STOP


    ## 뱀 객체 move() 메서드

    # 뱀 이동
    def move(self):
        cur = self.positions[0]
        x, y = self.direction
        new = (cur[0] + (x * GRID_SIZE)), (cur[1] + (y * GRID_SIZE))


        ## 뱀이 자기 몸통에 닿았을 때 처리 코드

        # 뱀이 자기 몸통에 닿았을 경우 뱀 처음부터 다시 생성
        if new in self.positions[2:]:
            self.die = True


        ## 뱀이 게임 화면을 넘어갈 때 처리 코드

        # 뱀이 게임 화면을 넘어갈 경우 뱀 처음부터 다시 생성
        elif new[0] < 0 or new[0] >= SCREEN_WIDTH or \
            new[1] < 0 or new[1] >= SCREEN_HEIGHT:
            self.die = True


        ## 뱀이 정상적으로 이동하는 경우 처리 코드

        # 뱀이 정상적으로 이동하는 경우
        else:
            self.positions.insert(0, new)
            if len(self.positions) > self.length:
                self.positions.pop()


    ## 뱀 객체 eat() 메서드

    # 뱀이 먹이를 먹을 때 호출
    def eat(self):
        self.length += 1


    ## 뱀 객체 draw() 메서드

    # 뱀 그리기
    def draw(self, screen):
        red, green, blue = 50 / (self.length - 1), 150, 150 / (self.length - 1)
        for i, p in enumerate(self.positions):
            color = (100 + red * i, green, blue * i)
            rect = pygame.Rect((p[0], p[1]), (GRID_SIZE, GRID_SIZE))
            pygame.draw.rect(screen, color, rect)
    
    


## 먹이 객체 코드

# 먹이 객체
class Feed():


    ## 먹이 객체 초기화 __init__() 메서드 코드

    def __init__(self):
        self.position = (0, 0)
        self.color = ORANGE
        self.create()


    ## 먹이 객체 create() 메서드 코드

    # 먹이 생성
    def create(self):
        x = random.randint(0, GRID_WIDTH - 1)
        y = random.randint(0, GRID_HEIGHT - 1)
        self.position = x * GRID_SIZE, y * GRID_SIZE


    ## 먹이 객체 draw() 메서드 코드

    # 먹이 그리기
    def draw(self, screen):
        rect = pygame.Rect((self.position[0], self.position[1]), (GRID_SIZE, GRID_SIZE))
        pygame.draw.rect(screen, self.color, rect)
    
    
## 스네이크 게임 객체 코드

# 게임 객체
class Game():
    

    ## 게임 객체 초기화 __init__() 메서드 코드
    def __init__(self):
        self.snake = Snake()
        self.feed = Feed()
        self.speed = 5
        
        
    ## 게임 이벤트 처리 process_events() 메서드 코드
    
    # 게임 이벤트 처리 및 조작
    def process_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return True
            elif event.type == pygame.KEYDOWN:
                if self.snake.die == True:
                    if event.key == pygame.K_RETURN:
                        self.snake.create()
                        self.feed.create()
                else:
                    if event.key == pygame.K_UP:
                        self.snake.control(UP)
                    elif event.key == pygame.K_DOWN:
                        self.snake.control(DOWN)
                    elif event.key == pygame.K_LEFT:
                        self.snake.control(LEFT)
                    elif event.key == pygame.K_RIGHT:
                        self.snake.control(RIGHT)
                    
        return False
    
    
    ## 게임 로직 수행 run_login() 메서드 코드
    
    # 게임 로직 수행
    def run_logic(self):
        self.snake.move()
        self.check_eat(self.snake, self.feed)
        self.speed = (20 + self.snake.length) / 4
        
        
    ## 먹이 체크 check_eat() 메서드 코드
    
    # 뱀이 먹이를 먹었는지 체크
    def check_eat(self, snake, feed):
        if snake.positions[0] == feed.position:
            snake.eat()
            feed.create()
            
    
    ## 게임 정보 표시 draw_info() 메서드 코드
    
    # 게임 정보 출력
    def draw_info(self, length, speed, screen):
        info = "Length: " + str(length) + "    " + "Speed: " + str(round(speed, 2))
        font = pygame.font.SysFont('FixedSys', 30, False, False)
        text_obj = font.render(info, True, GRAY)
        text_rect = text_obj.get_rect()
        text_rect.x, text_rect.y = 10, 10
        screen.blit(text_obj, text_rect)
    
    def draw_game_over(self, screen):
        
        font = pygame.font.SysFont("카페24 쑥쑥", 100, False, False)
        over_text_obj = font.render("GAME OVER", True, RED)
        over_text_rect = over_text_obj.get_rect()
        over_text_rect.x, over_text_rect.y = (SCREEN_WIDTH - over_text_rect[-2]) / 2, (SCREEN_HEIGHT - over_text_rect[-1]) / 2
        screen.blit(over_text_obj, over_text_rect)
        
        font = pygame.font.SysFont("카페24 쑥쑥", 30, False, False)
        enter_text_obj = font.render("press ENTER", True, RED)
        enter_text_rect = enter_text_obj.get_rect()
        enter_text_rect.x, enter_text_rect.y = over_text_rect[0] + over_text_rect[-2] - enter_text_rect[-2] - 10, \
                                               over_text_rect[1] + over_text_rect[-1] - enter_text_rect[-1] + 10
        screen.blit(enter_text_obj, enter_text_rect)
    
        
    ## 게임 프레임 처리 display_frame() 메서드 코드
    
    # 게임 프레임 처리
    def display_frame(self, screen):
        screen.fill(WHITE)
        self.snake.draw(screen)
        self.feed.draw(screen)
        self.draw_info(self.snake.length, self.speed, screen)
        if self.snake.die == True:
            self.draw_game_over(screen)
        screen.blit(screen, (0, 0))
        
        
## 스네이크 게임 메인 함수 main() 코드

def main():
    # 게임 초기화 및 환경 설정
    pygame.init()
    pygame.display.set_caption('Snake Game')
#     current_path = os.path.dirname('Snake_Game.py')
#     image_path = os.path.join(current_path, 'image')
#     image = pygame.image.load(os.path.join(image_path, 'snake.png'))
#     pygame.display.set_icon(image)
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    game = Game()


    ## 스네이크 게임 반복 처리 코드
    done = False
    while not done:
        done = game.process_events()
        game.run_logic()
        game.display_frame(screen)
        pygame.display.flip()
        clock.tick(game.speed)

    pygame.quit()


## 메인 함수 실행 코드 

if __name__ == '__main__':
    main()